### 8.1 python标准库的时间处理函数

#### 1. datetime模块
1. 当前时间 : `datetime.now`
2. datetime对象初始化 : `datetime(year,month,day,hour,minutes,seconds,miliseconds,timezone)`, 参数都是int型

In [1]:
from datetime import datetime 
print(datetime.now())
print(datetime(1991,7,31,11,20,2))

2018-06-25 09:52:11.815098
1991-07-31 11:20:02


#### 2. datetime与string相互转化
1. datetime转换为string  
  datetime.strftime(TIMEFORMAT) : 转换为对应格式的字符串  
   1. %Y
   2. %m
   3. %d
   4. %H
   5. %M
   6. %S
2. string转换为datetime对象  
   使用dateutil包, parser.parse(str)  
   该方法不用传入时间格式, 自动判断
  

In [2]:
# date2str
stmp = datetime(2018,5,19,11,59,0)
time2str = stmp.strftime('%Y-%m-%d %H:%M:%S')
print time2str
# str2date
from dateutil import parser
print parser.parse(time2str)


2018-05-19 11:59:00
2018-05-19 11:59:00


#### 3. pandas中转换datetime
1. pd.to_datetime(Series/DateFrame/Index)
2. pandas中时间空值: Nat , not a time

In [3]:
import pandas as pd
from pandas import Series,DataFrame,Index
pindex = Index(['2011-07-06 12:00:00', '2011-08-06 00:00:00','NaT'])

datetimeindex = pd.to_datetime(pindex)

print datetimeindex
print datetimeindex.isnull()

DatetimeIndex(['2011-07-06 12:00:00', '2011-08-06 00:00:00', 'NaT'], dtype='datetime64[ns]', freq=None)
[False False  True]


### 8.2 时间序列基础
#### 一. pandas中的时间索引
1. pandas中, 最基本的时间序列类型是以时间戳为索引的Series. 时间戳可以是python字符串或Datatime类型  
2. 两个时间索引的Series之间运算, 会自动按照时间对齐  
3. pandas使用numpy的datetime64数据类型以纳秒存储时间戳    
4. Datetime索引中每一项都是pandas的Timestamp类型    
 pandas.Timestamp对象可以随时转换为Datetime对象

In [24]:
# 1.时间序列
from datetime import datetime
import numpy as np
data = [datetime(2018,5,1),datetime(2018,5,12),datetime(2018,5,6)]
ts = pd.Series(np.random.rand(3),index=data)
print ts
print '============================'
print ts.index
# 2.时间序列之间的运算
print ts[::2] # 每隔2个取一项
print ts + ts[::2]
print '============================'
# 3.时间类型索引以纳秒存储
print ts.index.dtype
print type(ts.index)
print '============================'
# 4.时间索引的每一项都是Tiestamp
ts.index[0]

2018-05-01    0.423616
2018-05-12    0.289706
2018-05-06    0.348699
dtype: float64
DatetimeIndex(['2018-05-01', '2018-05-12', '2018-05-06'], dtype='datetime64[ns]', freq=None)
2018-05-01    0.423616
2018-05-06    0.348699
dtype: float64
2018-05-01    0.847233
2018-05-06    0.697397
2018-05-12         NaN
dtype: float64
datetime64[ns]
<class 'pandas.core.indexes.datetimes.DatetimeIndex'>


Timestamp('2018-05-01 00:00:00')

#### 二. 递增的时间索引产生
1. pd.date_range() 自动生成时间索引  
  1. 传入起始时间
  2. periods: 生成多少个时间索引
  3. freq: 生成索引的时间间隔, 默认是一天  
    1. D: 每日
    2. B: 每个工作日
    3. H: 每小时
    4. min: 每分钟
    5. M: 每月最后一个日历日
    6. BM: bussiness month, 每月最后一个工作日  
      ... 此外, 还有很多日历生成规则

2. 关于freq属性的其他用法  
 除了上面的字母标识外, 还有很多其他表示方法
  1. 可以直接传入频率字符串, 比如'3h25min'
  2. 'WOM': week of month, 是你能获得注入每月第三个星期五这类

In [56]:
# 每隔5小时产生一个索引
pd.date_range('2018/01/01',periods=10,freq='5H')

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 05:00:00',
               '2018-01-01 10:00:00', '2018-01-01 15:00:00',
               '2018-01-01 20:00:00', '2018-01-02 01:00:00',
               '2018-01-02 06:00:00', '2018-01-02 11:00:00',
               '2018-01-02 16:00:00', '2018-01-02 21:00:00'],
              dtype='datetime64[ns]', freq='5H')

In [68]:
pd.date_range('2018/01/01',periods=5,freq='3h50min')

DatetimeIndex(['2018-01-01 00:00:00', '2018-01-01 03:50:00',
               '2018-01-01 07:40:00', '2018-01-01 11:30:00',
               '2018-01-01 15:20:00'],
              dtype='datetime64[ns]', freq='230T')

In [69]:
pd.date_range('2018/01/01',periods=5,freq='WOM-3FRI')

DatetimeIndex(['2018-01-19', '2018-02-16', '2018-03-16', '2018-04-20',
               '2018-05-18'],
              dtype='datetime64[ns]', freq='WOM-3FRI')

#### 三. 索引选取
1. 传入表示时间的字符串  
2. 只传入'年','月'的字符串即可对时间序列数据分片  
3. 由于时间可以比大小, 因此可以在索引选取时指定时间范围: `start:end`    
4. 另一种选取时间范围的数据是 Series.truncate(after='',before='')  
  truncate返回的是原始Series的视图, 不改变原Series的数据, 因此也是一种切片操作

In [47]:
# 1. 时间的字符串选取
print ts['2018/05/01']
print '============================'
# 2. 按照'年'切片数据
longerts = pd.Series(np.random.rand(1000),index=pd.date_range('2000/1',periods=1000)) # periods:按天递增的个数
print longerts[-3:]
print ''
series_2001 = longerts['2001']
print series_2001[:3]
print '============================'
# 3.按照'月'切片数据
series_2001_05 = longerts['2000-05']
print series_2001_05[:3]


2018-05-01    0.423616
dtype: float64
2002-09-24    0.000534
2002-09-25    0.508329
2002-09-26    0.221663
Freq: D, dtype: float64

2001-01-01    0.541888
2001-01-02    0.901428
2001-01-03    0.184969
Freq: D, dtype: float64
2000-05-01    0.475697
2000-05-02    0.953077
2000-05-03    0.209156
Freq: D, dtype: float64


In [49]:
# 4. 指定时间范围
longerts['2000/5':'2000/6'].head()

2000-05-01    0.475697
2000-05-02    0.953077
2000-05-03    0.209156
2000-05-04    0.371667
2000-05-05    0.235582
Freq: D, dtype: float64

In [52]:
# truncate
longerts.truncate(after='2000-01-09')

2000-01-01    0.453443
2000-01-02    0.889495
2000-01-03    0.357315
2000-01-04    0.010386
2000-01-05    0.026101
2000-01-06    0.824207
2000-01-07    0.215657
2000-01-08    0.072001
2000-01-09    0.413610
Freq: D, dtype: float64

#### 四. 带有重复时间索引的数据
1. 检查索引的is_unique属性, 查看索引是否重复
2. 通过时间索引值来选取元素时, 要么返回标量, 要么返回切片, 关键看传入的索引是否是重复索引
3. 对重复索引聚合使用groupby方法, 传入level=0

In [59]:
dates = pd.DatetimeIndex(['1/1/2000', '1/2/2000', '1/2/2000','1/2/2000', '1/3/2000'])
dup_ts = pd.Series(np.arange(5),dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int64

In [62]:
# 返回False, 表示索引中存在重复元素
print dup_ts.index.is_unique
print '============================'
# 重复索引选取
print dup_ts['2000-01-02']
print '============================'
dup_ts.groupby(level=0).mean()

False
2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int64


2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int64

### 8.3 日期的范围,频率及移动

#### 一. 超前或滞后数据
1. sihft(n) : 整体数据向下移动  
 shift惭怍不会改变索引的位置, 只会改变column方向上的数据

2. 通过偏移量对日期位移  
  1. pandas.tseries.offset包含有日期偏移对象 : Day,MonthEnd    
  2. MonthEnd偏移的用法 :  
    1. MonthEnd(): 当月最后一天  
    2. MonthEnd(2): 后一个月最后一天   
    3. MonthEnd.rollforward(time)/rollback(time)明确向前/向后进行偏移
  
3. 通过groupby(fuc), 指定确定同一个分组使用的函数来聚合数据

In [72]:
ts = pd.Series(np.random.rand(4),
              index=pd.date_range('2000-01-01',periods=4,freq='M')) # 每月最后一天
ts

2000-01-31    0.598369
2000-02-29    0.707779
2000-03-31    0.669988
2000-04-30    0.024994
Freq: M, dtype: float64

In [75]:
# 1.shift偏移
print ts.shift(2)
ts.shift(-1)

2000-01-31         NaN
2000-02-29         NaN
2000-03-31    0.598369
2000-04-30    0.707779
Freq: M, dtype: float64


2000-01-31    0.707779
2000-02-29    0.669988
2000-03-31    0.024994
2000-04-30         NaN
Freq: M, dtype: float64

In [81]:
# 2. 偏移量移动时间
from pandas.tseries.offsets import Day,MonthEnd
now = datetime.now()
print 'now + 2*Day',now + 2*Day()
print 'now + MonthEnd',now + MonthEnd() 
print 'now + 2MonthEnd',now + MonthEnd(2)

print '============================'

# MonthEnd锚点偏移
offset = MonthEnd()
print 'offset.rollforward(now) : ',offset.rollforward(now)
print 'offset.rollback(now)',offset.rollback(now)

now + 2*Day 2018-06-27 11:54:52.799993
now + MonthEnd 2018-06-30 11:54:52.799993
now + 2MonthEnd 2018-07-31 11:54:52.799993
offset.rollforward(now) :  2018-06-30 11:54:52.799993
offset.rollback(now) 2018-05-31 11:54:52.799993


In [99]:
# 3. groupby(func)
ts = pd.Series(np.random.rand(10),
              index=pd.date_range('2000-01-01',periods=10,freq='5d'))
print ts
from pandas.tseries import offsets 
offset = offsets.MonthBegin()
ts.groupby(offset.rollforward).mean()

2000-01-01    0.929174
2000-01-06    0.233330
2000-01-11    0.446800
2000-01-16    0.835928
2000-01-21    0.020729
2000-01-26    0.685061
2000-01-31    0.413527
2000-02-05    0.584343
2000-02-10    0.180762
2000-02-15    0.411811
Freq: 5D, dtype: float64


2000-01-01    0.929174
2000-02-01    0.439229
2000-03-01    0.392305
dtype: float64

### 8.4 时区处理
#### 一. 何为时区
1. 以格林威治时间(UTC)为世界时间标准.
2. 时区以格林威治时间偏移量来表示
3. python的pytz包, 有关时区的信息都在这里  
  1. 获取所有时区: `pytz.common_timezones`
  2. 创建时区对象: `pytz.timezone`

In [110]:
# 查看前三个时区
import pytz
print pytz.common_timezones[:3]
print pytz.country_timezones('cn')
pytz.timezone('Asia/Shanghai')

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa']
[u'Asia/Shanghai', u'Asia/Urumqi']


<DstTzInfo 'Asia/Shanghai' LMT+8:06:00 STD>

#### 二. pandas时区本地化和转换
1. pandas中的时间索引, 默认不带时区
2. 将Series/DataFrame转变为带时区(本地化)的 : `Series/DataFrame.tz_localize('UTC..')`
3. 时区转换: `Series/DataFrame.tz_convert('')`
4. 上述两个方法, 也可以应用在DateIndex上

In [130]:
dateIdx = pd.date_range('1990-01-01 9:30',periods=6,freq='D')
ts = pd.Series(np.random.rand(6),index=dateIdx)
print ts.index.tz #None, 默认没有时区
print '不带时区:\n',ts 
ts_utc = ts.tz_localize('UTC')
print '带时区:\n', ts_utc
print '东八区\n', ts_utc.tz_convert('Asia/Shanghai')

None
不带时区:
1990-01-01 09:30:00    0.399953
1990-01-02 09:30:00    0.827621
1990-01-03 09:30:00    0.898261
1990-01-04 09:30:00    0.315004
1990-01-05 09:30:00    0.118807
1990-01-06 09:30:00    0.932909
Freq: D, dtype: float64
带时区:
1990-01-01 09:30:00+00:00    0.399953
1990-01-02 09:30:00+00:00    0.827621
1990-01-03 09:30:00+00:00    0.898261
1990-01-04 09:30:00+00:00    0.315004
1990-01-05 09:30:00+00:00    0.118807
1990-01-06 09:30:00+00:00    0.932909
Freq: D, dtype: float64
东八区
1990-01-01 17:30:00+08:00    0.399953
1990-01-02 17:30:00+08:00    0.827621
1990-01-03 17:30:00+08:00    0.898261
1990-01-04 17:30:00+08:00    0.315004
1990-01-05 17:30:00+08:00    0.118807
1990-01-06 17:30:00+08:00    0.932909
Freq: D, dtype: float64


#### 三. pandas的Timestamp对象也可以设置时区
1. 创建Timestamp对象 : `pd.Timestamp('time')`
2. pands的时间索引,无论带什么时区, 内部都按照UTC时区存储
2. 分配时区和更改时区:  
  `Timestamp.tz_convert`与`Timestamp.tz_localize`
3. offset对象与时间运算, 会保留时区
4. 不同时区索引的pandas数据结构进行运算时, 会按照UTC时间对齐索引, 并返回UTC类型的索引

In [157]:
stamp = pd.Timestamp('2011-09-09 11:20')
print stamp
shanghai_stamp = stamp.tz_localize('Asia/Shanghai')
print shanghai_stamp
print shanghai_stamp.tz_convert('UTC')
from pandas.tseries.offsets import Hour
print shanghai_stamp+2*Hour()
print '================================='
# 不同时区进行运算, 会转换为UTC时间对齐, 且返回UTC时间索引
ts = pd.Series(np.random.rand(5),index=pd.date_range('2018-01-01 12:30',periods=5,freq='B'))
ts1 = ts[:2].tz_localize('Europe/London')
ts2 = ts1[:2].tz_convert('Europe/Moscow')
print ts
print ts1+ts2

2011-09-09 11:20:00
2011-09-09 11:20:00+08:00
2011-09-09 03:20:00+00:00
2011-09-09 13:20:00+08:00
2018-01-01 12:30:00    0.292266
2018-01-02 12:30:00    0.573058
2018-01-03 12:30:00    0.576714
2018-01-04 12:30:00    0.405501
2018-01-05 12:30:00    0.296579
Freq: B, dtype: float64
2018-01-01 12:30:00+00:00    0.584532
2018-01-02 12:30:00+00:00    1.146117
Freq: B, dtype: float64


### 8.5 时期及算术运算

### 8.6 重采样及频率转换

### 8.7 移动窗口函数

In [83]:
ts.groupby??

In [ ]:
Series().groupby